# Classification of gHM in four types of terrestrial anthromes
This notebook has the methodology used to create the raster layers from the [GHM data](https://onlinelibrary.wiley.com/doi/abs/10.1111/gcb.14549). This is a single raster with a continuous variable from 0 to 1 (Low to high level of encroachment). The resolution of this dataset is 1-km. [Supplementary figure 11](https://onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1111%2Fgcb.14549&file=gcb14549-sup-0001-AppendixS1.pdf) from the gHM paper shows a summary (the median) of the GHm values accross 19 Anthromes). 
Anthrome data can be downloaded from [here](http://ecotope.org/anthromes/v2/data/) , [appendix 3](https://onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1111%2Fj.1466-8238.2010.00540.x&file=GEB_540_sm_AppS3.pdf) of the paper on Anthromes explains how the land has been classified.
Using this Anthrome Classification Algorithm, we will reclassify the anthromes using the following logic: 
```
Urban: 11: Urban/12: Mixed settlements
Rainfed agriculture: 21: Rice villages/23: Rainfed villages/32: Residential rainfed croplands/33: Populated croplands/34: Remote croplands
Irrigated agriculture: 22: Irrigated villages/31: Residential irrigated croplands
Rangeland: 24: Pastoral villages/41: Residential rangelands/42: Populated rangelands/43: Remote rangelands
Other: 51: Residential woodlands/52: Populated woodlands/53: Remote woodlands/54: Inhabited treeless and barren lands/61: Wild woodland
```

This would mean that we would have 4 gHM layersthat do not overlap:
- Urban
- Rainfed agriculture
- Irrigated agriculture
- Rangeland

Human pressure data was pre-processed by MOL 

**README file:**
```
Global human modification layer
2019.11.27
THESE FILES ARE FOR INTERNAL DESIGN PURPOSES ONLY, AND SHOULD NOT BE
ADVERTISED, USED, OR SHARED OUTSIDE OF THE HALF-EARTH TEAM UNTIL FURTHER
NOTICE.
gHM_extended_maskGADM36_cea_20191127.grd: 1km raster layer of global human
modification (gHM), derived from Kennedy et al. (2019). The original
layer was transformed to an equal-area projection using bilinear
interpolation. It was then extended to all terrestrial surface area,
filling in missing areas with a value of 0 (e.g., Antarctica). The
was then masked to the GADM3.6 land layer*.
Kennedy, C. M., Oakleaf, J. R., Theobald, D. M., Baruch‐Mordo, S., &amp;
Kiesecker, J. (2019). Managing the middle: A shift in conservation priorities
based on the global human modification gradient. Global Change Biology,
25(3), 811-826. Data available at:
https://figshare.com/articles/Global_Human_Modification/7283087/1
GADM version 3.6 available at:
https://gadm.org/download_world.html
```

In [1]:
import arcpy
from arcpy.sa import *
import datetime

In [2]:
arcpy.env.workspace = "C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\Human Pressure"

In [3]:
ghm = "gHM_extended_maskGADM36_cea_20191127.tif"
anthromes = "anthromes\\anthro2_a2000.tif"

In [14]:
ghm_cellx = arcpy.GetRasterProperties_management(ghm, "CELLSIZEX")
ghm_celly = arcpy.GetRasterProperties_management(ghm, "CELLSIZEY")
ghm_cell_size = f"{ghm_cellx.getOutput(0)} {ghm_celly.getOutput(0)}"

In [4]:
ghm_spatial_ref = arcpy.Describe(ghm).spatialReference
anth_spatial_ref = arcpy.Describe(anthromes).spatialReference

In [24]:
gdb = "Human_pressure_outputs.gdb"
anth_cea = f"{gdb}\\anthro2_a2000_cea"
anth_resample = f"{anth_cea}_Resample"
anth_reclass = f"{anth_resample}_Reclass"

The anthrome data has to be resampled to match the projection of gHM layer and the cell size. 

In [10]:
start = datetime.datetime.now()
arcpy.management.ProjectRaster(in_raster = anthromes, 
                               out_raster = anth_cea, 
                               out_coor_system = ghm_spatial_ref, 
                               resampling_type = "NEAREST", 
                               cell_size = None, 
                               geographic_transform = None, 
                               Registration_Point = None, 
                               in_coor_system = anth_spatial_ref, 
                               vertical = "NO_VERTICAL")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:09.157904


In [19]:
start = datetime.datetime.now()
arcpy.management.Resample(in_raster= anth_cea, 
                          out_raster= anth_resample, 
                          cell_size= ghm_cell_size, 
                          resampling_type= "NEAREST")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:01:30.200974


In [22]:
start = datetime.datetime.now()
reclass_raster = arcpy.sa.Reclassify(anth_resample, 
                                 "LABEL", 
                                 "'11: Urban' 1;'12: Mixed settlements' 1;'21: Rice villages' 2;'22: Irrigated villages' 3;'23: Rainfed villages' 2;'24: Pastoral villages' 4;'31: Residential irrigated croplands' 3;'32: Residential rainfed croplands' 2;'33: Populated croplands' 2;'34: Remote croplands' 2;'41: Residential rangelands' 4;'42: Populated rangelands' 4;'43: Remote rangelands' 4;'51: Residential woodlands' 0;'52: Populated woodlands' 0;'53: Remote woodlands' 0;'54: Inhabited treeless and barren lands' 0;'61: Wild woodlands' 0;'62: Wild treeless and barren lands' 0", "DATA");
reclass_raster.save(f"..\\{anth_reclass}") #this does not use the env parameter, it uses the location of the notebook as root
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:22.165552


In [23]:
reclass_dict = {
    "Urban": "1",
    "Rainfed": "2",
    "Irrigated": "3",
    "Rangeland": "4"
}

In [26]:
for i, j in reclass_dict.items():
    start = datetime.datetime.now()
    out_name = f"..\\{gdb}\\gHM_{i}"
    outSetNull = SetNull(reclass_raster, ghm, f"VALUE <> {j}")
    outSetNull.save(out_name)  #this does not use the env parameter, it uses the location of the notebook as root
    end = datetime.datetime.now() - start
    print(f"{i} finished in {end}")

Urban finished in 0:01:51.781387
Rainfed finished in 0:03:33.917372
Irrigated finished in 0:01:49.571088
Rangeland finished in 0:01:56.986422
